In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *
from sloppy.metrics import *
from ipywidgets import Layout, IntSlider, FloatLogSlider, FloatSlider, interactive, fixed
import inspect

In [3]:
import cma

In [4]:
#def LensCavCAD(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, lens_diam=6.35, lens_thick=4., Rlens=5.0):
#    return LensCav(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, lens_diam=lens_diam, lens_thick=lens_thick, Rlens=Rlens)
def LensCavCADpert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, c4=6e-4, c6=0., c8=0., eps=np.zeros((12,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    quart_thick=2.
    quart_dist=1e-6
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    pa = np.array([lens_dist/2.+lens_thick/2.+quart_dist, 0, 0])
    pb = np.array([lens_dist/2.+lens_thick/2.+quart_dist+quart_thick,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    pc = np.array([-lens_dist/2.-lens_thick/2.-quart_dist-quart_thick,0,0])
    pd = np.array([-lens_dist/2.-lens_thick/2.-quart_dist,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])

    ps = np.stack([p0,p1,pa,pb,p2,p3,p4,p5,pc,pd,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
        
    #apply poition perturbations
    ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]    
        
    hi = 12.7
    qi= 7.75
    ng = 1.4537
    coef = np.zeros(9)
    coef[4] = c4
    coef[6] = c6
    coef[8] = c8
    elements = [\
                Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                FreeFormInterface(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=lens_diam, n2=ng, coef=coef),\
                Glass(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=lens_diam, n1=ng),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                Mirror(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=hi),\
                Mirror(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=hi),\
                Glass(p=ps[8], n=ns[8], ax=ax_x[8], ay=ax_y[8], Rbasis=Rtr[8], diameter=lens_diam, n2=ng),\
                FreeFormInterface(p=ps[9], n=ns[9], ax=ax_x[9], ay=ax_y[9], Rbasis=Rtr[9], diameter=lens_diam, n1=ng, coef=-coef),\
                CurvedGlass(p=ps[10], n=ns[10], ax=ax_x[10], ay=ax_y[10], Rbasis=Rtr[10], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[11], n=ns[11], ax=ax_x[11], ay=ax_y[11], Rbasis=Rtr[11], diameter=lens_diam, n1=ng)\
                ]
    return elements

cavfct = LensCavCADpert
parname = 'lens_dist'

In [5]:
def LensCavPert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, eps=np.zeros((8,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])
    ps = np.stack([p0,p1,p2,p3,p4,p5,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
    
    #apply poition perturbations
    ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]   
        
    hi = 12.7
    qi=7.75
    ng = 1.4537
     #negative sign of firstt cuved surface for abcd matrix
    elements = [Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                Mirror(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=hi),\
                Mirror(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=hi),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                CurvedGlass(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=lens_diam, n1=ng)]
    return elements

cavfct = LensCavPert
parname = 'lens_dist'

In [6]:
res = degeneracy_length(cavfct, parname, 5e-2, s=1, degmodenum=2)
Ldeg=res.x
print(Ldeg)

21.77801052565133


In [7]:
waists_vs_param(cavfct, parname, 5e-2, s=1, degmodenum=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

21.780501672240803

In [8]:
def get_eigenray_traj(cavfct, cavpars, mu=[0., 0., 0., 0.], Nrt=3, Niter=10, Niter_inner=50, lr=0.03, lr_decay=1.0):
    elements = cavfct(**cavpars)
    sys = RaySystem(elements)
    #ray0 = np.stack([np.atleast_2d(sys.screen.p+np.array([0, 1e-3,0])), np.atleast_2d(sys.screen.n)], axis=0)
    ray0 = sys.screen.eigenvectors_to_rays(mu) #TODO
    reig3, tols = sys.find_eigenray_mpe_dev(ray0, Nrt=Nrt, lr=lr, Niter=Niter_inner, Nmpe=Niter, get_tols=True, lr_decay=lr_decay)
    traj = sys.propagate(reig3, Nrt=Nrt)
    return traj, tols

In [9]:
traj0, tols0 = get_eigenray_traj(cavfct, {parname: Ldeg}, mu=[0, 0, 0, 0], Nrt=1, lr=1e-4, Niter=3)

In [10]:
def get_eigenray_cma(cavfct, cavpars, mu=[0., 0., 0., 0.], Nrt=1, sigma=0.1, verb_disp=100):
    elements = cavfct(**cavpars)
    sys = RaySystem(elements)
    
    def loss(x, sys, Nrt=1):
        rcur = sys.screen.eigenvectors_to_rays(np.asarray(x))
        traj = sys.propagate(rcur, Nrt=Nrt)
        rnew = traj[-1,...]
        return np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()
    
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    es = cma.CMAEvolutionStrategy(mu, sigma)
    es.optimize(loss, args=(sys, Nrt), verb_disp=verb_disp)
    mueig = es.result.xbest
    reig3 = sys.screen.eigenvectors_to_rays(mueig)
    traj = sys.propagate(reig3, Nrt=Nrt)
    return traj, [es.result.fbest,]

In [11]:
eps=np.zeros((8,5))
eps[0:2,1] = 1e-3
traj2, tols2 = get_eigenray_cma(cavfct, {parname: Ldeg, 'eps': eps}, mu=[0, 0, 0, 0], Nrt=1, verb_disp=100)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=357616, Thu Oct  8 20:23:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.727487053074964e-01 1.0e+00 9.25e-02  8e-02  1e-01 0:00.0
    2     16 1.366983559327899e-01 1.2e+00 8.83e-02  8e-02  9e-02 0:00.0
    3     24 1.254636106895493e-01 1.3e+00 8.01e-02  7e-02  8e-02 0:00.0
  100    800 5.282841189452758e-06 1.2e+02 2.92e-04  5e-05  9e-05 0:00.2
  178   1424 1.853903751561144e-06 1.9e+02 3.96e-07  5e-09  8e-09 0:00.4
  178   1424 1.853903751561144e-06 1.9e+02 3.96e-07  5e-09  8e-09 0:00.4
termination by {'tolfun': 1e-11}
best f-value = 1.8539037406074172e-06
solution = [ 0.0316697   0.01513088  0.01276856 -0.02654883]


In [12]:
try:
    plot.close()
except:
    pass
elements = cavfct(**{parname: Ldeg})
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)
    #plot += k3d.points(el.p, color=0xffff00, point_size=0.5)
    
plot_trajs(traj0, plot)
#plot_trajs(traj1, plot, color=0x00ff00)
plot_trajs(traj2, plot, color=0xff0000)
plot += k3d.points(traj0[:,0,0,:], color=0xff0000, point_size=0.3)

plot.display()

Output()

In [118]:
def get_perturbation_one(cavfct, cavpars, which, ax, rng, mu=[0, 0, 0, 0], Nrt=3, lr=0.01, Niter=20, relative=True, get_tols=False):
    #get reference trajectory
    traj0, tols0 = get_eigenray_traj(cavfct, cavpars, mu=mu, Nrt=Nrt)
    
    #move one element and find perturbed eigenray trajectory
    N = rng.shape[0]
    alltraj = np.zeros((N,traj0.shape[0],3))
    alltols = np.zeros(N)
    for i, e in enumerate(rng):
        eps = np.zeros((8,5))
        eps[which, ax] += e
        pars = cavpars.copy()
        pars['eps'] = eps
        try:
            #traj1, tols1 = get_eigenray_traj(cavfct, pars, mu=mu, Nrt=Nrt, lr=lr, Niter=Niter, Niter_inner=50, lr_decay=0.5) #get_eigenray_traj(cavfct, pars, mu=mu, Nrt=Nrt, lr=lr)
            traj1, tols1 = get_eigenray_cma(cavfct, pars, mu=mu, Nrt=Nrt, verb_disp=False)
        except:
            alltraj[i,...] = np.nan
            alltols[i] = np.nan
        else:
            alltraj[i,...] = traj1[:,0,0,:]
            alltols[i] = tols1[-1]
        
    if relative:
        if get_tols:
            return np.linalg.norm(traj0[None,:,0,0,:]-alltraj, axis=-1), alltols
        else:
            return np.linalg.norm(traj0[None,:,0,0,:]-alltraj, axis=-1)
    else:
        if get_tols:
            return alltraj, alltols
        else:
            return alltraj

In [152]:
def get_perturbation_ray(cavfct, cavpars, which, ax, rng, mu=[0, 0, 0, 0], Nrt=3, get_tols=False):
    #get reference trajectory
    traj0, tols0 = get_eigenray_traj(cavfct, cavpars, mu=mu, Nrt=Nrt)
    
    #move one element and find perturbed eigenray trajectory
    N = rng.shape[0]
    alltraj = np.zeros((N,traj0.shape[0],2,3))
    alltols = np.zeros(N)
    for i, e in enumerate(rng):
        eps = np.zeros((8,5))
        eps[which, ax] += e
        pars = cavpars.copy()
        pars['eps'] = eps
        try:
            traj1, tols1 = get_eigenray_cma(cavfct, pars, mu=mu, Nrt=Nrt, verb_disp=False)
        except:
            alltraj[i,...] = np.nan
            alltols[i] = np.nan
        else:
            alltraj[i,...] = traj1[:,:,0,:]
            alltols[i] = tols1[-1]
        
    if get_tols:
        return alltraj, alltols
    else:
        return alltraj

In [13]:
def get_eigenray_cma_sys(cavfct, cavpars, mu=[0., 0., 0., 0.], Nrt=1, sigma=0.1, verb_disp=100):
    elements = cavfct(**cavpars)
    sys = RaySystem(elements)
    
    def loss(x, sys, Nrt=1):
        rcur = sys.screen.eigenvectors_to_rays(np.asarray(x))
        traj = sys.propagate(rcur, Nrt=Nrt)
        rnew = traj[-1,...]
        return np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()
    
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    es = cma.CMAEvolutionStrategy(mu, sigma)
    es.optimize(loss, args=(sys, Nrt), verb_disp=verb_disp)
    mueig = es.result.xbest
    reig3 = sys.screen.eigenvectors_to_rays(mueig)
    traj = sys.propagate(reig3, Nrt=Nrt)
    return sys, mueig, traj, [es.result.fbest,]

def get_perturb_ray_sys(cavfct, cavpars, which, ax, rng, mu=[0, 0, 0, 0], Nrt=3):
    #get reference trajectory
    sys0, mu0, traj0, tols0 = get_eigenray_cma_sys(cavfct, cavpars, mu=mu, Nrt=Nrt, verb_disp=False)
    
    #move one element and find perturbed eigenray trajectory
    N = rng.shape[0]
    alltraj = np.zeros((N,traj0.shape[0],2,3))
    alltols = np.zeros(N)
    allmus = np.zeros((N,4))
    allsys = []
    for i, e in enumerate(rng):
        eps = np.zeros((8,5))
        eps[which, ax] += e
        pars = cavpars.copy()
        pars['eps'] = eps
        try:
            sys1, mu1, traj1, tols1 = get_eigenray_cma_sys(cavfct, pars, mu=mu, Nrt=Nrt, verb_disp=False)
            allsys.append(sys1)
        except:
            alltraj[i,...] = np.nan
            allmus[i,...] = np.nan
            alltols[i] = np.nan
        else:
            alltraj[i,...] = traj1[:,:,0,:]
            alltols[i] = tols1[-1]
            allmus[i,...] = mu1
    return allsys, allmus, alltraj, alltols

In [14]:
rng = np.linspace(-50e-3, 50e-3, 50)
allsys, allmus, rays_anim, tols = get_perturb_ray_sys(cavfct, cavpars={parname: Ldeg}, Nrt=1, which=[0,1], ax=1, rng=rng)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=287961, Thu Oct  8 20:24:21 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=227707, Thu Oct  8 20:24:28 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=335894, Thu Oct  8 20:24:32 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=341248, Thu Oct  8 20:24:35 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=332646, Thu Oct  8 20:24:40 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=350848, Thu Oct  8 20:24:43 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=302092, Thu Oct  8 20:24:47 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=282238, Thu Oct  8 20:24:51 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=305564, Thu Oct  8 20:24:55 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=270421, Thu Oct  8 20:24:59 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=323244, Thu Oct  8 20:25:03 2020)
(4_w,8)-aC

In [15]:
plt.figure()
plt.semilogy(rng, tols)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
Nel = len(allsys[0].elements)
mmatrices = [{str(t/100): s.elements[i].plot().model_matrix for t, s in enumerate(allsys)} for i in range(Nel)]

In [149]:
from joblib import dump

In [150]:
savedict = {'range': rng, 'trajectories': traj_anim}
dump(savedict, 'trajectories_y_20um.p')

['trajectories_y_20um.p']

In [17]:
traj_anim = rays_anim[:,:,0,:] #get only positions for plotting 
traj_anim_ts = traj_to_timeseries(traj_anim[:,:,None,:], step=1)

In [78]:
#create more round trips for each solution for plotting
traj_anim = []
for i, sys in enumerate(allsys):
    traj = sys.propagate(sys.screen.eigenvectors_to_rays(allmus[i]), Nrt=20)
    traj_anim.append(traj[:,0,0,:])
traj_anim = np.stack(traj_anim, axis=0)
traj_anim_ts = traj_to_timeseries(traj_anim[:,:,None,:], step=1)

In [18]:
elements = allsys[0].elements

In [19]:
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    mesh = el.plot(opacity=0.4)
    mesh.model_matrix = mmatrices[i]
    plot += mesh

#plot_trajs(traj, plot)
for ts in traj_anim_ts:
    plot += k3d.line(ts, shader='mesh', width=0.05, color=0x00ff00)

plot.display()

Output()

## Metrics
- Lens curved surface separation
- Lens curved surface AOI
- Lens flat surface AOI
- Displacement / tilt of cavity axis at the input mirror i.e., which modes get excited by a fixed input beam?

In [87]:
traj_anim[0,:,:]

array([[ 0.00000000e+00,  8.51343849e-08, -2.59509521e-07],
       [ 8.88900526e+00,  2.02035662e-06,  3.75358457e-07],
       [ 1.28890053e+01,  2.61940635e-06,  5.71882396e-07],
       [ 2.40000013e+01,  2.39746108e-06,  7.88870192e-07],
       [ 9.43980876e+00,  1.89999997e+01,  1.16571894e+01],
       [-9.43980392e+00,  1.89999989e+01, -1.16571902e+01],
       [-2.39999991e+01, -2.39885969e-06, -7.84607492e-07],
       [-1.28890053e+01, -2.44913780e-06, -1.09090063e-06],
       [-8.88900526e+00, -1.85008797e-06, -8.94376860e-07],
       [ 0.00000000e+00,  8.51345840e-08, -2.59509439e-07]])

In [20]:
lens_separation = np.linalg.norm(traj_anim[:,2,:] - traj_anim[:,-3,:], axis=-1)
plt.figure()
plt.plot(rng*1e3, lens_separation)
plt.xlabel('Lens displacement [um]')
plt.ylabel('Curved surface separation [mm]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
input_displace = np.linalg.norm(traj_anim[:,-4,:] - traj0[-4,0,0,:], axis=-1)
plt.figure()
plt.plot(rng*1e3, input_displace)
plt.xlabel('Lens displacement [um]')
plt.ylabel('Ray displacement @ Input mirror  [mm]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
aoi_flat1 = np.arccos(np.dot(rays_anim[:,0,1,:], -elements[0].n))
plt.figure()
plt.plot(rng*1e3, np.rad2deg(aoi_flat1))
plt.xlabel('Lens displacement [um]')
plt.ylabel('AOI @ flat surface 1 [deg]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
aoi_flat2 = np.arccos(np.dot(rays_anim[:,-2,1,:], -elements[7].n)) # -3: after curved glass, 7: last flat glass
plt.figure()
plt.plot(rng*1e3, np.rad2deg(aoi_flat2))
plt.xlabel('Lens displacement [um]')
plt.ylabel('AOI @ flat surface 2 [deg]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
cps = np.stack([s.elements[1].jopt.cp for s in allsys], axis=0)

In [25]:
n = cps[0,:] - rays_anim[0,2,0,:] # [0,2,0,:] position on curved glass surface
n = n/np.linalg.norm(n)

In [26]:
ns = cps[0,:] - rays_anim[:,2,0,:] # [:,2,0,:] position on curved glass surface
ns = ns/np.linalg.norm(ns, axis=-1)[:,None]

In [27]:
aoi_curv1 = np.arccos(np.einsum('ni,ni->n', rays_anim[:,2,1,:], -ns)) #TODO catch: curved lens poition also moves!!! Find a good way to do that...

In [28]:
plt.figure()
plt.plot(rng*1e3, np.rad2deg(aoi_curv1))
plt.xlabel('Lens displacement [um]')
plt.ylabel('AOI @ curved surface 1 [deg]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Optimization Test

In [12]:
from scipy.optimize import minimize, dual_annealing, basinhopping

In [13]:
eps=np.zeros((8,5))
eps[0:2,1] = 1e-3
elements = cavfct(**{parname: Ldeg, 'eps': eps})
sys = RaySystem(elements)

In [14]:
x = np.array([0., 0., 0., 0.])
rcur = sys.screen.eigenvectors_to_rays(x)
traj = sys.propagate(rcur, Nrt=1)
rnew = traj[-1,...]

In [62]:
rnew

array([[[ 3.38020349e-08, -1.00932977e-04,  1.43981782e-04]],

       [[ 9.99999891e-01,  2.67781338e-04, -3.81992439e-04]]])

In [35]:
np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()

0.0010545920467091205

In [85]:
np.linalg.norm(rcur-rnew)

0.0007566473753843225

In [49]:
def loss(x, sys, Nrt=1):
    rcur = sys.screen.eigenvectors_to_rays(np.asarray(x))
    traj = sys.propagate(rcur, Nrt=Nrt)
    rnew = traj[-1,...]
    return np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()

In [50]:
loss(x, sys)

0.0027953745195005058

In [59]:
a = 5e-3
N = 100
xx = np.linspace(-a, a, 50)
x, y = np.meshgrid(xx, xx)
losses = np.empty_like(x)

In [60]:
for i in range(len(xx)):
    for j in range(len(xx)):
        losses[i, j] = loss(np.array([5e-4, 0., x[i,j], y[i,j]]), sys)

In [61]:
plt.figure()
plt.pcolormesh(x, y, losses)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
res = minimize(loss, x0=np.array([0., 0., 0., 0.]), args=(sys,), method='L-BFGS-B', bounds=[(-1, 1), (-1, 1), (-0.5, 0.5), (-0.5, 0.5)]) 

In [53]:
#res = minimize(loss, x0=(0., 0., 0., 0.), args=(sys,), method='trust-constr', bounds=[(-1, 1), (-1, 1), (-0.5, 0.5), (-0.5, 0.5)]) 

In [63]:
res = minimize(loss, x0=np.array([0., 0., -5e-4, 1e-4]), args=(sys,), method='BFGS')

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen


In [64]:
res

      fun: nan
 hess_inv: array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])
      jac: array([nan, nan, nan, nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 672
      nit: 1
     njev: 112
   status: 2
  success: False
        x: array([-310.362662  , -587.8101473 ,  700.66896752, -369.95790256])

In [65]:
lw = [-0.5] * 4
up = [0.5] * 4
ret = dual_annealing(loss, bounds=list(zip(lw, up)), args=(sys, ), seed=1234, maxiter=10**4)

In [66]:
ret

     fun: nan
 message: ['Maximum number of iteration reached']
    nfev: 351894
    nhev: 0
     nit: 10000
    njev: 0
  status: 0
 success: True
       x: array([ 0.27997581, -0.22740739, -0.22353574,  0.30187218])

In [75]:
es = cma.CMAEvolutionStrategy(4 * [0], 0.1)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1010753, Mon Oct  5 18:44:28 2020)


In [77]:
es.optimize(loss, args=(sys,), verb_disp=100)

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.500504133740355e-01 1.0e+00 8.00e-02  7e-02  8e-02 0:00.0
    2     16 1.372318804411779e-01 1.2e+00 7.18e-02  6e-02  7e-02 0:00.0
    3     24 7.923790957990179e-02 1.3e+00 6.84e-02  6e-02  7e-02 0:00.0
  100    800 3.363774002143986e-06 1.5e+02 2.44e-04  4e-05  7e-05 0:00.2
  172   1376 1.853903724680585e-06 3.1e+02 1.51e-07  2e-09  4e-09 0:00.4
  172   1376 1.853903724680585e-06 3.1e+02 1.51e-07  2e-09  4e-09 0:00.4
termination by {'tolfun': 1e-11}
best f-value = 1.8539037212523853e-06
solution = [ 0.0316697   0.01513088  0.01276856 -0.02654883]


In [84]:
mueig = es.result.xbest
reig2 = sys.screen.eigenvectors_to_rays(mueig)

In [113]:
xopt, es = cma.fmin2(loss, 4 * [0], 0.1, args=(sys,), n_jobs=8)

TypeError: fmin() got an unexpected keyword argument 'n_jobs'

In [87]:
traj2 = sys.propagate(reig2)

In [89]:
try:
    plot.close()
except:
    pass
elements = cavfct(**{parname: Ldeg})
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)
    
plot_trajs(traj0, plot)
plot_trajs(traj1, plot, color=0x00ff00)
plot_trajs(traj1, plot, color=0xff0000)

plot += k3d.points(traj0[:,0,0,:], color=0xff0000, point_size=0.3)

plot.display()

Output()

In [151]:
traj2.shape

(10, 2, 1, 3)

In [46]:
def rays_interaction_factory_pert(cavfct, cavpars, rmax=6.):
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(11,6.6), sharex='row', sharey='row')
    ms = 8.
    lines = [a.scatter([], [], c=[], cmap='jet', s=ms) for a in ax.flatten()]
    ax[0,0].set_title('Raytracing')
    ax[0,1].set_title('ABCD')
    ax[0,2].set_title('ABCD (Extracted)')
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')

    def makeidx(hit_m):
        return np.arange(hit_m.shape[0])
    
    magnitude = lambda x: 1. if x==0. else int(np.floor(np.log10(x)))
    
    def update(ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        #kwargs.update({parname: Ldeg+dl*1e-1})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0] #fix for now since waist is not at 0 anymore!
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_scrs = sys.screen.s_to_screen_coords(traj_hit[:,1,0,:])

        lines[0].set_offsets(hit_scr[:,:])
        lines[0].set_array(makeidx(hit_scr))
        lines[3].set_offsets(hit_scrs[:,:])
        lines[3].set_array(makeidx(hit_scrs))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lines[1].set_offsets(hit_m[:,0:2])
        lines[1].set_array(makeidx(hit_m))
        lines[4].set_offsets(hit_m[:,2:])
        lines[4].set_array(makeidx(hit_m))
        
        #abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        #hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        #lines[2].set_offsets(hit_fd[:,0:2])
        #lines[2].set_array(makeidx(hit_fd))
        #lines[5].set_offsets(hit_fd[:,2:])
        #lines[5].set_array(makeidx(hit_fd))
        
        for l in lines:
            l.set_clim(vmin=0, vmax=Nrt)
            
        ax[0,0].set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax[0,0].set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        ax[1,0].set_xlim(1.5*np.nanmin(hit_scrs[:,0]), 1.5*np.nanmax(hit_scrs[:,0]))
        ax[1,0].set_ylim(1.5*np.nanmin(hit_scrs[:,1]), 1.5*np.nanmax(hit_scrs[:,1]))
        fig.canvas.draw_idle()

    lo = Layout(width='80%', height='30px')

    sliders = {v.name: FloatSlider(value=v.default, min=v.default*0.1, max=v.default*1.5, step=10**(magnitude(v.default)-3), readout_format='.2e', layout=lo) for v in inspect.signature(cavfct).parameters.values() if v.name not in cavpars.keys()}
    ars = FloatSlider(value=0.1, min=0., max=rmax, step=1e-2, readout_format='.3f', layout=lo)
    brs = FloatSlider(value=0.1, min=0., max=rmax, step=1e-2, readout_format='.3f', layout=lo)
    aps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    bps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    Nrts = IntSlider(value=100, min=100, max=2000, step=100, layout=lo)
    raysliders = {'ar': ars, 'br': brs, 'ap': aps, 'bp': bps, 'Nrt': Nrts}
    sliders.update(**raysliders)
    cavparsfixed = {k: fixed(v) for k, v in cavpars.items()}
    sliders.update(**cavparsfixed)
    #print(sliders)
    return interactive(update, **sliders)

In [47]:
eps = np.zeros((12,5))
eps[3,3] += 1e-1
rays_interaction_factory_pert(cavfct, cavpars={parname: Ldeg, 'eps': eps}, rmax=10.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='ar', layout=Layout(height='30px', width='80%'), max=…

In [34]:
metrics_interaction_factory(cavfct, cavpars={parname: Ldeg, 'c4': 6e-4, 'c6': 0, 'c8': 0}, rmax=12., Nplt=100, slrng=(0.8, 1.2), degmodenum=1, a=8., b=0., s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'lens_dist': 1, 'c4': -4, 'c6': 1.0, 'c8': 1.0}


interactive(children=(FloatSlider(value=8.0, description='a', layout=Layout(height='30px', width='80%'), max=1…

In [35]:
dct = {"a": 1, "b": 2}

In [36]:
for k,v in dct.items():
    print(k,v)

a 1
b 2


In [17]:
res = degeneracy_length(cavfct, parname, 1e-2)
Ldeg=res.x
sys = RaySystem(cavfct(lens_dist=Ldeg))

try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)
plot.display()

Output()

In [19]:
system = sys.abcd
print(system.Ltot)
system.waist_at(0)
mu1, mu2 = system.q

131.24995092745596


In [20]:
system.waist_at(0.)

array([0.0182912, 0.0182912])

In [21]:
mu = 1e-1*np.real(mu1) + 1e-3*np.real(mu2) 
ray = sys.screen.eigenvectors_to_rays(mu)

traj = sys.propagate(ray, Nrt=30)

In [22]:
screen = Screen(p=sys.screen.p, n=sys.screen.n, ax=sys.screen.ax, ay=sys.screen.ay, diameter=3.)

In [23]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements[:-1]):
    plot += el.plot(opacity=0.4)

plot += screen.plot(opacity=0.4)

plot_trajs(traj, plot, shader='mesh', width=0.02, color=0xc61920)
plot.display()

Output()